In [1]:
import sqlite3
import pandas as pd
import quandl
from datetime import datetime

# Set your Quandl API key
quandl.ApiConfig.api_key = 'FMezMmR86K7axszB_rkz'

# Define the series codes
series_codes = {"YIELD":'YIELD',"REALYIELD":'REALYIELD'}

# Prefix to be added to each series code
prefix = "USTREASURY/"

# Frequencies to download (daily, weekly, monthly)
frequencies = ['daily', 'weekly', 'monthly']

# Get the current date and time
current_datetime = datetime.now()
# Extract the date part
current_date = current_datetime.date()

# Create an SQLite connection and cursor
conn = sqlite3.connect('Treasury_Yields_data.db')
cursor = conn.cursor()

# Fetch data and create tables for each frequency
for series_code in series_codes:
    for frequency in frequencies:
        full_series_code = prefix + series_code

        # Adjust the frequency in the Quandl request
        data = quandl.get(full_series_code, collapse=frequency)
        
        # Remove any observations that represent "accrued" observations
        data = data[data.index <= pd.to_datetime(current_date)]
            
        # Create a table for the original series
        table_name = f"{series_codes[series_code]}_{frequency}"
        # Create a table for the original series
        data.to_sql(table_name, conn, if_exists='replace', index=True)

        # Create a table for the period-to-period difference
        diff_data = data.diff()
        diff_table_name = f"{table_name}_diff"
        diff_data.to_sql(diff_table_name, conn, if_exists='replace', index=True)

        # Create a table for the percent change
        percent_change_data = data.pct_change()
        percent_change_table_name = f"{table_name}_percent_change"
        percent_change_data.to_sql(percent_change_table_name, conn, if_exists='replace', index=True)

# Commit changes and close connection
conn.commit()
conn.close()

print("Databases created successfully.")


Databases created successfully.


In [6]:
# correct missing data in weekly treasury datasets
# 2017-04-16 00:00:00 is all zeroes

import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('Treasury_Yields_data.db')

# Create a cursor object
cursor = conn.cursor()

# Specify the date and the new values for each column
date_to_update = '2017-04-16 00:00:00'  # Replace with the actual date
new_values = {
    '[10 YR]': '2.30',
    '[2 YR]': '1.25',
    '[1 YR]': '1.05',
    '[5 YR]': '1.83',
    '[30 YR]': '2.93',
    '[20 YR]': '2.67',
    '[3 MO]': '0.82',
    '[1 MO]': '0.76',
    '[3 YR]': '1.45',
    '[6 MO]': '0.95',
    '[7 YR]': '2.16'
    # Add more columns and values as needed
}

# Execute the update statement for each column
for col, value in new_values.items():
    update_query = f"UPDATE YIELD_weekly SET {col} = ? WHERE Date = ?"
    cursor.execute(update_query, (value, date_to_update))

# Commit the changes
conn.commit()

# Close the cursor and the database connection
cursor.close()
conn.close()

In [27]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('Treasury_Yields_data.db')

# Specify the date for which to calculate differences
date_to_update = '2017-04-09 00:00:00'  # Replace with the actual date

# Fetch the three observations (before, on, and after the specified date) from YIELD_weekly
query = f"SELECT * FROM YIELD_weekly WHERE Date >= ? ORDER BY Date LIMIT 3"
df = pd.read_sql_query(query, conn, params=(date_to_update,))

# Convert relevant columns to numeric
numeric_cols = ['10 YR', '2 YR', '1 YR', '5 YR', '30 YR', '20 YR', '3 MO', '1 MO', '3 YR', '6 MO', '7 YR']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Drop columns with NaN values
df = df.dropna(axis=1)

# Convert remaining columns to numeric
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Calculate the differences
df_diff = pd.concat([df[['Date']], df[numeric_cols].diff().dropna()], axis=1)

# Calculate the percentage changes
df_percent_change = pd.concat([df[['Date']], (df_diff[numeric_cols] / df[numeric_cols].shift()).dropna() * 100], axis=1)

# Display the results if needed
print("Original DataFrame:")
print(df)
print("\nDifferences:")
print(df_diff)
print("\nPercentage Changes:")
print(df_percent_change)

# Close the database connection
conn.close()


Original DataFrame:
                  Date  1 MO  3 MO  6 MO  1 YR  2 YR  3 YR  5 YR  7 YR  10 YR  \
0  2017-04-09 00:00:00  0.77  0.82  0.95  1.08  1.29  1.52  1.92  2.20   2.38   
1  2017-04-16 00:00:00  0.76  0.82  0.95  1.05  1.25  1.45  1.83  2.16   2.30   
2  2017-04-23 00:00:00  0.72  0.79  0.92  0.99  1.20  1.40  1.77  2.05   2.24   

   20 YR  30 YR  
0   2.74   3.00  
1   2.67   2.93  
2   2.61   2.89  

Differences:
                  Date  10 YR  2 YR  1 YR  5 YR  30 YR  20 YR  3 MO  1 MO  \
0  2017-04-09 00:00:00    NaN   NaN   NaN   NaN    NaN    NaN   NaN   NaN   
1  2017-04-16 00:00:00  -0.08 -0.04 -0.03 -0.09  -0.07  -0.07  0.00 -0.01   
2  2017-04-23 00:00:00  -0.06 -0.05 -0.06 -0.06  -0.04  -0.06 -0.03 -0.04   

   3 YR  6 MO  7 YR  
0   NaN   NaN   NaN  
1 -0.07  0.00 -0.04  
2 -0.05 -0.03 -0.11  

Percentage Changes:
                  Date     10 YR      2 YR      1 YR      5 YR     30 YR  \
0  2017-04-09 00:00:00       NaN       NaN       NaN       NaN       NaN   

In [34]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('Treasury_Yields_data.db')

# Specify the dates for which to update the tables
dates_to_update = df_diff['Date'].iloc[-2:]

# Update 'YIELD_weekly_diff' and 'YIELD_weekly_percent_change' tables for each date
for date in dates_to_update:
    # Update 'YIELD_weekly_diff' table
    difference_table = 'YIELD_weekly_diff'
    for col in df_diff.columns[1:]:
        update_query_diff = f'UPDATE {difference_table} SET "{col}" = ? WHERE Date = ?'
        cursor = conn.cursor()
        cursor.execute(update_query_diff, (df_diff[col].loc[df_diff['Date'] == date].iloc[0], date))
        conn.commit()

    # Update 'YIELD_weekly_percent_change' table
    percent_change_table = 'YIELD_weekly_percent_change'
    for col in df_percent_change.columns[1:]:
        update_query_percent_change = f'UPDATE {percent_change_table} SET "{col}" = ? WHERE Date = ?'
        cursor = conn.cursor()
        cursor.execute(update_query_percent_change, (df_percent_change[col].loc[df_percent_change['Date'] == date].iloc[0], date))
        conn.commit()

# Close the database connection
conn.close()


